In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [2]:
train = pd.read_csv('DataSets/train.csv')
test = pd.read_csv('DataSets/test.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#removendo colunas que são desnecessaria para o nosso modelo
train.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [5]:
#Criando novos dfs a partir do One-Hot enconding (dummies)
new_data_train = pd.get_dummies(train)
new_data_test = pd.get_dummies(test)

In [6]:
new_data_train.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [7]:
new_data_test.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,0,1,0,1,0
1,893,3,47.0,1,0,7.0000,1,0,0,0,1
2,894,2,62.0,0,0,9.6875,0,1,0,1,0
3,895,3,27.0,0,0,8.6625,0,1,0,0,1
4,896,3,22.0,1,1,12.2875,1,0,0,0,1


In [8]:
new_data_train.isnull().sum().sort_values(ascending=False).head(10)

Age           177
Embarked_S      0
Embarked_Q      0
Embarked_C      0
Sex_male        0
Sex_female      0
Fare            0
Parch           0
SibSp           0
Pclass          0
dtype: int64

In [9]:
#Preenchendo os valores null na coluna Age
new_data_train['Age'].fillna(new_data_train['Age'].mean(), inplace=True)
new_data_test['Age'].fillna(new_data_test['Age'].mean(), inplace=True)

In [10]:
new_data_test.isnull().sum().sort_values(ascending=False).head(10)

Fare          1
Embarked_S    0
Embarked_Q    0
Embarked_C    0
Sex_male      0
Sex_female    0
Parch         0
SibSp         0
Age           0
Pclass        0
dtype: int64

In [11]:
new_data_test['Fare'].fillna(new_data_test['Fare'].mean(), inplace=True)

In [12]:
#Separando features e target para criação do modelo
x = new_data_train.drop('Survived', axis=1)
y = new_data_train['Survived']

In [13]:
#Criando no modelo
tree = DecisionTreeClassifier(max_depth=3, random_state=0)
tree.fit(x, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [14]:
#Verificando o score do conjunto de treino
tree.score(x,y)

0.8271604938271605

In [16]:
submission = pd.DataFrame()
submission["PassengerId"] = new_data_test["PassengerId"]
submission['Survived'] = tree.predict(new_data_test)

In [17]:
submission.to_csv('submission.csv', index=False)